# Marche descendante: comparaison des méthodes numériques


In [ ]:
from trustutils import run 
from trustutils.jupyter import plot
import numpy as np
from math import *
import matplotlib.pyplot as plt
import os
from functions import *

run.introduction("M. El Moatamid","28/06/2023")
run.description(description_fiche)

run.TRUST_parameters()

## Préparation des calculs
Cette cellule est à remplir par l'utilisateur :

In [ ]:
# liste des config disponible = ["VEF_k-epsilon", "VDF_k-epsilon","polymac_k-omega", "polymac_k-tau", "vdf_k-omega", "vdf_k-tau"]
# l'ordre des config n'est pas important !
#config = ["VEF_k-epsilon", "polymac_k-omega" ,"vdf_k-omega"]
config = ["VEF_k-epsilon", "polymac_k-omega" ,"vdf_k-omega", "polymac_k-tau"]

# # Dimensions du domaine (expérience de Driver & Seegmiller [1])
h = 0.0127
H = 8*h
L = 0.5
l = 1.1143
print(f"Re_h = {44.2*1.17*h/(1.98e-5):.2e}")

# # Fluide 
mu = 1.469e-5
rho = 1

# # Calcul parallèle
number_of_partitions = 4

# # Maillage : chaque valeur dans les listes suivantes correspond à un maillage
# Nombre de MAILLES selon x et y (du rectangle sans la marche)
Nx = [132, 198] # de préférence un multiple de 6
Ny = [48, 84] # de préférence un multiple de 12

# Inflations
growth_rate_x_inlet = [0.98, 0.98]
growth_rate_y_inlet = [1.05, 1.05]
growth_rate_x_step = [1.03, 1.04]
growth_rate_y_step = [1.01, 1.01]

# # force le calcul (suppression des anciens résultats)
force_recalculation = True # put True to force recalculation

# # répertoire de travail
build = run.BUILD_DIRECTORY # build directory (absolute path !)

## Lancement des calculs

In [ ]:
# # Initialisation et calcul
nb_method = len(config)
nb_mesh = len(Nx)
mesh_param = [Nx, Ny, growth_rate_x_inlet, growth_rate_y_inlet, growth_rate_x_step, growth_rate_y_step]
fluid_param = [mu, rho]

# # substitution dictionnaries
dict_list = substitution(config,mesh_param,fluid_param)
    
if force_recalculation or not os.path.exists(build):
    run.reset() # Delete the build directory and empty the list of cases to be executed
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
    for i in range(nb_method):
        file = "marche_pb_multi"
        if "k-epsilon" in config[i]:
            file = f"marche_{config[i]}"
        for j in range(nb_mesh):
            if number_of_partitions == 1:
                run.addCaseFromTemplate(f"{file}.data",directory=build+f"/{config[i]}/mesh_{j}",d=dict_list[i][j])
            else:
                os.system(f"mkdir -p {build}/{config[i]}")
                os.system(f"mkdir -p {build}/{config[i]}/mesh_{j}")
                GenerateInputFile(f"{build}/{config[i]}/mesh_{j}",build,dict_list[i][j], file, config[i])
                os.system(f'cd {build}/{config[i]}/mesh_{j};  make_PAR.data marche_{config[i]} {number_of_partitions}; cd ../../..')
                run.addCase(f"{config[i]}/mesh_{j}", f"PAR_marche_{config[i]}.data",nbProcs=number_of_partitions)
    run.printCases()
    run.runCases() # lance les différents cas test écrits dans build/
    perf = run.tablePerf() # tableau des performances de calcul
else:
    print("Using previous calculation")

### Evolution des résidus

In [ ]:
# Residuals Plot
n_col = 2 # nombre de colomnes
fig = plt.figure(figsize=(15,ceil(nb_method/n_col)*4))
axs = fig.subplots(ceil(nb_method/n_col),n_col)


for i in range(nb_method):
    file = f"PAR_marche_{config[i]}"
    for j in range(nb_mesh):
        res_file = build+f"/{config[i]}/mesh_{j}/{file}.dt_ev"
        max_residu = plot.loadText(res_file)[3,:]
        # find what is the max residual (called res_name)
        res_name = ""
        for k, res in enumerate(plot.loadText(res_file)[4:,-1]):
            if max_residu[-1] == res:
                res_name = get_residu_name(res_file,k)
        temps = plot.loadText(res_file)[0,:]
        axs[int(i/n_col),i%n_col].plot(temps, max_residu,label=f"mesh_{j}")
        axs[int(i/n_col),i%n_col].text(0.3, 0.3-j*0.1, f'mesh_{j} : {res_name}', horizontalalignment='center', verticalalignment='center', transform=axs[int(i/n_col),i%n_col].transAxes)
    axs[int(i/n_col),i%n_col].legend(fontsize=10)
    axs[int(i/n_col),i%n_col].set_xlabel("time", fontsize=10)
    axs[int(i/n_col),i%n_col].set_ylabel("maxResidu", fontsize=10)
    axs[int(i/n_col),i%n_col].set_yscale('log')
    axs[int(i/n_col),i%n_col].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Residuals")
fig.tight_layout()

décommenter pour voir les champs de vitesse

In [ ]:
from trustutils import visit
#visit.showMesh(build+f"/{config[i]}/marche_{config[i]}.lata","dom")
#for i in range(len(config)):
#    fig=visit.Show(build+f"/{config[i]}/mesh_1/PAR_marche_{config[i]}.lata","Pseudocolor","VITESSE_SOM_dom_magnitude")
#    fig.plot()

### Statistiques des calculs

In [ ]:
# Tableau des performances de calcul
import pandas as pd

performance = open(build+"/perf.csv", "w")
for i in range(nb_method):
    file = f"PAR_marche_{config[i]}"
    for j in range(nb_mesh):
        performance.write(read_perf(build+f"/{config[i]}/mesh_{j}/{file}.perf", file, config[i]+f" mesh_{j}"))
performance.close() 
performance = open(build+"/perf.csv", "r")
data = pd.read_csv(performance, sep=" ", header=None)
df = data[data.columns[:-1]]
df.columns = ["Configuration", "Mesh", "Host", "System", "total_CPU_time", "CPU time/step", "nb_cells"]
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

In [ ]:
grpd = df.groupby('Configuration')
for name, data in grpd:
    plt.plot(data.nb_cells.values, data.total_CPU_time.values, 'o-', label = name)
plt.legend()
plt.xlabel("number of cells")
plt.ylabel("Total CPU time [s]")
plt.title("Calculation time")

## Post-traitement des résultats
### Longueur de recirculation
Les profils de vitesse selon x (axe de l'écoulement) sont tracés en y=0 (bas de la marche). La longueur de recirculation correspond à l'abscisse pour laquelle s'annule. De ce fait, la vitesse en bas de la marche est négative dans la zone de recirculation, avant de redevenir positive plus loin.

In [ ]:
Graph=plot.Graph("Profil de vitesse en y=0 m", nX=nb_mesh, size=[15,8])
for j in range(nb_mesh):
    Graph.addPlot(j,f"mesh_{j}")
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_PAROI"
        Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}.son", linestyle="-",compo=0,label=config[i])
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
#Graph.label("y [m]", "u [m/s]")

On calcule la longueur de recirculation en cherchant x tel que u(x)=0. Cette longueur est normalisé avec la hauteur de la marche, puis comparée à la valeur trouvée expérimentalement.

In [ ]:
# On écrit les longueur de recirculation calculées dans un csv
recirculation_length = open(build+"/recirculation_length.csv", "w")
ref_val = 0.0774/h # valeur experimentale de l/h
# On parcours l'ensemble des configurations et maillages réalisés
for i in range(nb_method):
    for j in range(nb_mesh):
        file = f"PAR_marche_{config[i]}_SONDE_U_PAROI"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])
        except:
            u = np.zeros(100)
        x, _ = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        x_min, x_max = 0.01, 0.3 # define interval to search for the function's zero (x | u(x)=0)
        l_recirculation = find_zero(x,u,x_min,x_max) # recirculation length : find x such as u(x)=0
        recirculation_length.write(f"{config[i]},mesh_{j},{l_recirculation/h:.2f},{int(100*(l_recirculation/h-ref_val)/ref_val)}%\n")
recirculation_length.write(f"Driver's experiment, ,{ref_val:.2f}, \n") # ref value
recirculation_length.close() 
f = open(build+"/recirculation_length.csv", "r")
df = pd.read_csv(f, sep=",", header=None)
df.columns = ["Configuration", "mesh", "recirculation length (l/h)", "relative gap"]
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

### Comparaison des profils de vitesse en différentes positions aux données expériemntales

In [ ]:
fig = plt.figure(figsize=(15,nb_mesh*6))
axs = fig.subplots(nb_mesh)

for j in range(nb_mesh):
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_X-4H"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])/44.2
        except:
            u = np.zeros(80)
        #y = np.linspace(0,0.1143,80)/h
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        y = y/h
        axs[j].plot(u,y,label=config[i])
        
    # plot reference data : experiment
    df = pd.read_csv(f"{build}/x-4h.csv", sep=" ", error_bad_lines=False)
    axs[j].scatter(df.iloc[:, 2], df.iloc[:, 1], marker="x", color="red", label="reference")
    # plot simulation data from turbmodels
    turbmodels = ["wilcox2006"]
    for model in turbmodels:
        df = pd.read_csv(f"{build}/../data/{model}/{model}_x-4h.csv", sep=" ", error_bad_lines=False)
        axs[j].plot(df.iloc[:, 1], df.iloc[:, 0], linestyle="dashed", label="wilcox2006 (TMR)")
    axs[j].legend()
    axs[j].set_xlabel("u/u_inlet")
    axs[j].set_ylabel("y/h")
    axs[j].set_title(f"Mesh_{j} : Profils de vitesse en x=-4h")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,nb_mesh*6))
axs = fig.subplots(nb_mesh)

for j in range(nb_mesh):
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_X1H"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])/44.2
        except:
            u = np.zeros(80)
        #y = np.linspace(0,0.1143,80)/h
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        y = y/h
        axs[j].plot(u,y,label=config[i])
        
    # plot reference data : experiment
    df = pd.read_csv(f"{build}/x1h.csv", sep=" ", error_bad_lines=False)
    axs[j].scatter(df.iloc[:, 2], df.iloc[:, 1], marker="x", color="red", label="reference")
    # plot simulation data from turbmodels
    for model in turbmodels:
        df = pd.read_csv(f"{build}/../data/{model}/{model}_x1h.csv", sep=" ", error_bad_lines=False)
        axs[j].plot(df.iloc[:, 1], df.iloc[:, 0], linestyle="dashed", label=f"{model} (TMR)")
    axs[j].legend()
    axs[j].set_xlabel("u/u_inlet")
    axs[j].set_ylabel("y/h")
    axs[j].set_title(f"Mesh_{j} : Profils de vitesse en x=1h")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,nb_mesh*6))
axs = fig.subplots(nb_mesh)

for j in range(nb_mesh):
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_X4H"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])/44.2
        except:
            u = np.zeros(80)
        #y = np.linspace(0,0.1143,80)/h
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        y = y/h
        axs[j].plot(u,y,label=config[i])
        
    # plot reference data : experiment
    df = pd.read_csv(f"{build}/x4h.csv", sep=" ", error_bad_lines=False)
    axs[j].scatter(df.iloc[:, 2], df.iloc[:, 1], marker="x", color="red", label="reference")
    # plot simulation data from turbmodels
    for model in turbmodels:
        df = pd.read_csv(f"{build}/../data/{model}/{model}_x4h.csv", sep=" ", error_bad_lines=False)
        axs[j].plot(df.iloc[:, 1], df.iloc[:, 0], linestyle="dashed", label=f"{model} (TMR)")
    axs[j].legend()
    axs[j].set_xlabel("u/u_inlet")
    axs[j].set_ylabel("y/h")
    axs[j].set_title(f"Mesh_{j} : Profils de vitesse en x=4h")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,nb_mesh*6))
axs = fig.subplots(nb_mesh)

for j in range(nb_mesh):
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_X6H"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])/44.2
        except:
            u = np.zeros(80)
        #y = np.linspace(0,0.1143,80)/h
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        y = y/h
        axs[j].plot(u,y,label=config[i])
        
    # plot reference data : experiment
    df = pd.read_csv(f"{build}/x6h.csv", sep=" ", error_bad_lines=False)
    axs[j].scatter(df.iloc[:, 2], df.iloc[:, 1], marker="x", color="red", label="reference")
    # plot simulation data from turbmodels
    for model in turbmodels:
        df = pd.read_csv(f"{build}/../data/{model}/{model}_x6h.csv", sep=" ", error_bad_lines=False)
        axs[j].plot(df.iloc[:, 1], df.iloc[:, 0], linestyle="dashed", label=f"{model} (TMR)")
    axs[j].legend()
    axs[j].set_xlabel("u/u_inlet")
    axs[j].set_ylabel("y/h")
    axs[j].set_title(f"Mesh_{j} : Profils de vitesse en x=6h")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15,nb_mesh*6))
axs = fig.subplots(nb_mesh)

for j in range(nb_mesh):
    for i in range(nb_method):
        file = f"PAR_marche_{config[i]}_SONDE_U_X10H"
        try:
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}.son")[1::2,1])/44.2
        except:
            u = np.zeros(80)
        #y = np.linspace(0,0.1143,80)/h
        x,y = get_coord_sonde(build+f"/{config[i]}/mesh_{j}/{file}.son")
        y = y/h
        axs[j].plot(u,y,label=config[i])
        
    # plot reference data : experiment
    df = pd.read_csv(f"{build}/x10h.csv", sep=" ", error_bad_lines=False)
    axs[j].scatter(df.iloc[:, 2], df.iloc[:, 1], marker="x", color="red", label="reference")
    # plot simulation data from turbmodels
    for model in turbmodels:
        df = pd.read_csv(f"{build}/../data/{model}/{model}_x10h.csv", sep=" ", error_bad_lines=False)
        axs[j].plot(df.iloc[:, 1], df.iloc[:, 0], linestyle="dashed", label=f"{model} (TMR)")
    axs[j].legend()
    axs[j].set_xlabel("u/u_inlet")
    axs[j].set_ylabel("y/h")
    axs[j].set_title(f"Mesh_{j} : Profils de vitesse en x=10h")

fig.tight_layout()